# Bioimage Model Zoo Example notebook

This notebook shows how to interact with the `bioimgaeio.core` programmatically to explore, load, and export content from the [BioImage Model Zoo](https://bioimage.io)."

## 0. Activate human readable output error messages and load dependencies

In [6]:
from bioimageio.spec.pretty_validation_errors import (
    enable_pretty_validation_errors_in_ipynb,
)
enable_pretty_validation_errors_in_ipynb()

from pathlib import Path
from ruyaml import YAML
from imageio import imread
import pooch
import json


import bioimageio.core
from bioimageio.spec.utils import download


#import hashlib
#import os

# we use napari for visualising images, you can install it via `pip install napari` or`conda install napari`
#import napari
#import numpy as np
#import xarray as xr

#import onnx
#import torch
#import matplotlib.pyplot as plt



# helper function for showing multiple images in napari
def show_images(*images, names=None):
    v = napari.Viewer()
    for i, im in enumerate(images):
        name = None if names is None else names[i]
        if isinstance(im, str):
            im = imageio.imread(im)
        v.add_image(im, name=name)


## 1. Load a model

### 1.1 Inspect available models in the Bioimage Model Zoo

In [2]:
yaml = YAML(typ="safe")

COLLECTION_URL = "https://raw.githubusercontent.com/bioimage-io/collection-bioimage-io/gh-pages/collection.json"

collection_path = Path(pooch.retrieve(COLLECTION_URL, known_hash=None))

with collection_path.open() as f:

    collection = json.load(f)

model_urls = [entry["rdf_source"] for entry in collection["collection"] if entry["type"] == "model"]
model_rdfs = [yaml.load(Path(pooch.retrieve(mu, known_hash=None))) for mu in model_urls]

print("All models checked.")

# nickname_list = []
print('List of models in BioImageIO:\n')
for model in model_rdfs:
    # nickname_list.append(model['config']['bioimageio']['nickname'])
    print(f"{model['name']}\n - {model['config']['bioimageio']['nickname']}\n - {model['config']['bioimageio']['doi']}")

All models checked.
List of models in BioImageIO:

NucleiSegmentationBoundaryModel
 - affable-shark
 - 10.5281/zenodo.6647674
StarDist H&E Nuclei Segmentation
 - chatty-frog
 - 10.5281/zenodo.6338615
LiveCellSegmentationBoundaryModel
 - hiding-tiger
 - 10.5281/zenodo.6647688
Neuron Segmentation in EM (Membrane Prediction)
 - impartial-shrimp
 - 10.5281/zenodo.5874742
Pancreatic Phase Contrast Cell Segmentation (U-Net)
 - discreet-rooster
 - 10.5281/zenodo.8186255
EnhancerMitochondriaEM2D
 - hiding-blowfish
 - 10.5281/zenodo.6811922
MitochondriaEMSegmentationBoundaryModel
 - kind-seashell
 - 10.5281/zenodo.6630266
Cell Segmentation from Membrane Staining for Plant Tissues
 - humorous-owl
 - 10.5281/zenodo.5888237
PlatynereisEMnucleiSegmentationBoundaryModel
 - organized-badger
 - 10.5281/zenodo.6028098
StarDist Fluorescence Nuclei Segmentation
 - fearless-crab
 - 10.5281/zenodo.6348085
B. Sutilist bacteria segmentation - Widefield microscopy - 2D UNet
 - placid-llama
 - 10.5281/zenodo.7

### 1.2 Load model from the BioImage Model Zoo

In [3]:
BMZ_MODEL_ID = "affable-shark" 
BMZ_MODEL_DOI = "" 
BMZ_MODEL_URL = "" 

# Load the model description
# ------------------------------------------------------------------------------

if BMZ_MODEL_ID != "":
    model = bioimageio.core.load_description(BMZ_MODEL_ID)  # TODO: load from bioimageio id
    print(f"The model '{model.name}' with ID '{BMZ_MODEL_ID}' has been correctly loaded.")
elif BMZ_MODEL_DOI != "":
    model = bioimageio.core.load_description(BMZ_MODEL_DOI)  # TODO: load from bioimageio id
    print(f"The model '{model.name}' with DOI '{BMZ_MODEL_DOI}' has been correctly loaded.")
elif BMZ_MODEL_URL != "":
    model = bioimageio.core.load_description(BMZ_MODEL_URL)  # TODO: load from bioimageio id
    print(f"The model '{model.name}' with URL '{BMZ_MODEL_URL}' has been correctly loaded.")
else:
    print('Please specify a model ID, DOI or URL')

2024-05-09 19:05:22.177 | INFO     | bioimageio.spec._internal.io_utils:open_bioimageio_yaml:115 - 🦈 loading affable-shark 1 from https://uk1s3.embassy.ebi.ac.uk/public-datasets/bioimage.io/affable-shark/1/files/rdf.yaml


The model 'NucleiSegmentationBoundaryModel' with ID 'affable-shark' has been correctly loaded.


In [7]:
print(f"The model '{model.name}' had the following properties and metadata")
print()
print(f" Description {model.description}")
print()
print(f" The authors of the model are {model.authors}")
print(f" and it is maintained by: {model.maintainers}")
print(f" License: {model.license}")
print()

print(f" If you use this model, you are expected to cite {model.cite}")
print()
print(f" Further documentation can be found here: {model.cite}")
print()
print(f" GitHub repository: {model.git_repo}")
print()
print(f"Covers of the model '{model.name}'")
for cover in model.covers:
    cover_data = imread(download(cover).path)
    plt.figure(figsize=(10, 10))
    plt.imshow(cover_data)
    plt.xticks([])
    plt.yticks([])
    plt.show()


The model 'NucleiSegmentationBoundaryModel' had the following properties and metadata

 Description Nucleus segmentation for fluorescence microscopy

 The authors of the model are [Author(affiliation='EMBL Heidelberg', email=None, orcid=None, name='Constantin Pape', github_user=None)]
 and it is maintained by: [Maintainer(affiliation=None, email=None, orcid=None, name='Constantin Pape', github_user='constantinpape')]
 License: CC-BY-4.0

 If you use this model, you are expected to cite [CiteEntry(text='training library', doi=None, url='https://doi.org/10.5281/zenodo.5108853'), CiteEntry(text='architecture', doi=None, url='https://doi.org/10.1007/978-3-319-24574-4_28'), CiteEntry(text='segmentation algorithm', doi=None, url='https://doi.org/10.1038/nmeth.4151'), CiteEntry(text='data', doi=None, url='https://www.nature.com/articles/s41592-019-0612-7')]

 Further documentation can be found here: [CiteEntry(text='training library', doi=None, url='https://doi.org/10.5281/zenodo.5108853'), C

/var/folders/_d/q8wfkdk54p70jv5jwwp1yrtd9px3jt/T/ipykernel_64081/3290842921.py:18: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  cover_data = imread(download(cover).path)


ImportError: dlopen(/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/PIL/_imaging.cpython-312-darwin.so, 0x0002): Library not loaded: '@rpath/libtiff.5.dylib'
  Referenced from: '/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/PIL/_imaging.cpython-312-darwin.so'
  Reason: tried: '/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/PIL/../../../libtiff.5.dylib' (no such file), '/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/PIL/../../../libtiff.5.dylib' (no such file), '/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/bin/../lib/libtiff.5.dylib' (no such file), '/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/bin/../lib/libtiff.5.dylib' (no such file), '/usr/local/lib/libtiff.5.dylib' (no such file), '/usr/lib/libtiff.5.dylib' (no such file)

In [8]:
model = bioimageio.core.load_description_and_test('affable-shark')
model.validation_summary.display()

2024-05-09 16:58:08.286 | INFO     | bioimageio.spec._internal.io_utils:open_bioimageio_yaml:115 - 🦈 loading affable-shark 1 from https://uk1s3.embassy.ebi.ac.uk/public-datasets/bioimage.io/affable-shark/1/files/rdf.yaml


❌ bioimageio validation: failed

source: https://uk1s3.embassy.ebi.ac.uk/public-datasets/bioimage.io/affable-shark/1/files/rdf.yaml
|     package     |  version   |
|       ---       |    ---     |
| bioimageio.spec | 0.5.2post4 |
| bioimageio.core | 0.6.3      |


|  ❓  |                  location                 |                                                                                                                                                                                            detail                                                                                                                                                                                           |
| --- |                    ---                    |                                                                                                                                                                                             ---                                                                                                                                                                                             |
| ✔️  |                                           | initialized model 0.4.10                                                                                                                                                                                                                                                                                                                                                                    |
| ✔️  |                                           | bioimageio.spec format validation model 0.4.10                                                                                                                                                                                                                                                                                                                                              |
| ⚠   | `weights.pytorch_state_dict.dependencies` | Custom dependencies (conda:environment.yaml) specified. Avoid this whenever possible to allow execution in a wider range of software environments.                                                                                                                                                                                                                                          |
|     |                                           |                                                                                                                                                                                                                                                                                                                                                                                             |
| ❌   |                                           | Reproduce test outputs from test inputs                                                                                                                                                                                                                                                                                                                                                     |
| ❌   | `weights`                                 | "batch_norm" not implemented for 'Byte'                                                                                                                                                                                                                                                                                                                                                     |
|     |                                           | [_resource_tests.py](_resource_tests.py "/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/bioimageio/core/_resource_tests.py"), line 129, in _test_model_inference<br>`    results = prediction_pipeline.predict_sample_without_blocking(inputs)`<br>`              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^`                      |
|     |                                           | [_prediction_pipeline.py](_prediction_pipeline.py "/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/bioimageio/core/_prediction_pipeline.py"), line 160, in predict_sample_without_blocking<br>`    self._adapter.forward(`                                                                                                                           |
|     |                                           | [_pytorch_model_adapter.py](_pytorch_model_adapter.py "/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/bioimageio/core/model_adapters/_pytorch_model_adapter.py"), line 67, in forward<br>`    result = self._network(  # pyright: ignore[reportUnknownVariableType]`<br>`             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^` |
|     |                                           | [module.py](module.py "/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/torch/nn/modules/module.py"), line 1532, in _wrapped_call_impl<br>`    return self._call_impl(*args, **kwargs)`<br>`           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^`                                                                                                              |
|     |                                           | [module.py](module.py "/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/torch/nn/modules/module.py"), line 1541, in _call_impl<br>`    return forward_call(*args, **kwargs)`<br>`           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^`                                                                                                                            |
|     |                                           | [d9e1780d938d2ad512b15843d55363a2-unet.py](d9e1780d938d2ad512b15843d55363a2-unet.py "/Users/marianaferreira/Library/Caches/bioimageio/d9e1780d938d2ad512b15843d55363a2-unet.py"), line 206, in forward<br>`    return self._apply_default(x)`<br>`           ^^^^^^^^^^^^^^^^^^^^^^`                                                                                                        |
|     |                                           | [d9e1780d938d2ad512b15843d55363a2-unet.py](d9e1780d938d2ad512b15843d55363a2-unet.py "/Users/marianaferreira/Library/Caches/bioimageio/d9e1780d938d2ad512b15843d55363a2-unet.py"), line 169, in _apply_default<br>`    x, encoder_out = self.encoder(x)`<br>`                     ^^^^^^^^^^^^^^^`                                                                                           |
|     |                                           | [module.py](module.py "/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/torch/nn/modules/module.py"), line 1532, in _wrapped_call_impl<br>`    return self._call_impl(*args, **kwargs)`<br>`           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^`                                                                                                              |
|     |                                           | [module.py](module.py "/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/torch/nn/modules/module.py"), line 1541, in _call_impl<br>`    return forward_call(*args, **kwargs)`<br>`           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^`                                                                                                                            |
|     |                                           | [d9e1780d938d2ad512b15843d55363a2-unet.py](d9e1780d938d2ad512b15843d55363a2-unet.py "/Users/marianaferreira/Library/Caches/bioimageio/d9e1780d938d2ad512b15843d55363a2-unet.py"), line 265, in forward<br>`    x = block(x)`<br>`        ^^^^^^^^`                                                                                                                                          |
|     |                                           | [module.py](module.py "/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/torch/nn/modules/module.py"), line 1532, in _wrapped_call_impl<br>`    return self._call_impl(*args, **kwargs)`<br>`           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^`                                                                                                              |
|     |                                           | [module.py](module.py "/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/torch/nn/modules/module.py"), line 1541, in _call_impl<br>`    return forward_call(*args, **kwargs)`<br>`           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^`                                                                                                                            |
|     |                                           | [d9e1780d938d2ad512b15843d55363a2-unet.py](d9e1780d938d2ad512b15843d55363a2-unet.py "/Users/marianaferreira/Library/Caches/bioimageio/d9e1780d938d2ad512b15843d55363a2-unet.py"), line 384, in forward<br>`    return self.block(x)`<br>`           ^^^^^^^^^^^^^`                                                                                                                          |
|     |                                           | [module.py](module.py "/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/torch/nn/modules/module.py"), line 1532, in _wrapped_call_impl<br>`    return self._call_impl(*args, **kwargs)`<br>`           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^`                                                                                                              |
|     |                                           | [module.py](module.py "/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/torch/nn/modules/module.py"), line 1541, in _call_impl<br>`    return forward_call(*args, **kwargs)`<br>`           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^`                                                                                                                            |
|     |                                           | [container.py](container.py "/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/torch/nn/modules/container.py"), line 217, in forward<br>`    input = module(input)`<br>`            ^^^^^^^^^^^^^`                                                                                                                                                     |
|     |                                           | [module.py](module.py "/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/torch/nn/modules/module.py"), line 1532, in _wrapped_call_impl<br>`    return self._call_impl(*args, **kwargs)`<br>`           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^`                                                                                                              |
|     |                                           | [module.py](module.py "/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/torch/nn/modules/module.py"), line 1541, in _call_impl<br>`    return forward_call(*args, **kwargs)`<br>`           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^`                                                                                                                            |
|     |                                           | [instancenorm.py](instancenorm.py "/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/torch/nn/modules/instancenorm.py"), line 87, in forward<br>`    return self._apply_instance_norm(input)`<br>`           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^`                                                                                                         |
|     |                                           | [instancenorm.py](instancenorm.py "/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/torch/nn/modules/instancenorm.py"), line 36, in _apply_instance_norm<br>`    return F.instance_norm(`<br>`           ^^^^^^^^^^^^^^^^`                                                                                                                            |
|     |                                           | [functional.py](functional.py "/opt/homebrew/Caskroom/miniconda/base/envs/bioimageiocore/lib/python3.12/site-packages/torch/nn/functional.py"), line 2553, in instance_norm<br>`    return torch.instance_norm(`<br>`           ^^^^^^^^^^^^^^^^^^^^`                                                                                                                                       |
|     |                                           |                                                                                                                                                                                                                                                                                                                                                                                             |


In [10]:
model = bioimageio.core.load_description('affable-shark')
model.validation_summary.display()

2024-05-09 16:58:41.586 | INFO     | bioimageio.spec._internal.io_utils:open_bioimageio_yaml:115 - 🦈 loading affable-shark 1 from https://uk1s3.embassy.ebi.ac.uk/public-datasets/bioimage.io/affable-shark/1/files/rdf.yaml


✔️ bioimageio validation: passed

source: https://uk1s3.embassy.ebi.ac.uk/public-datasets/bioimage.io/affable-shark/1/files/rdf.yaml
|     package     |  version   |
|       ---       |    ---     |
| bioimageio.spec | 0.5.2post4 |


|  ❓  |                  location                 |                                                                       detail                                                                       |
| --- |                    ---                    |                                                                        ---                                                                         |
| ✔️  |                                           | initialized model 0.4.10                                                                                                                           |
| ✔️  |                                           | bioimageio.spec format validation model 0.4.10                                                                                                     |
| ⚠   | `weights.pytorch_state_dict.dependencies` | Custom dependencies (conda:environment.yaml) specified. Avoid this whenever possible to allow execution in a wider range of software environments. |
|     |                                           |                                                                                                                                                    |


In [11]:
model = bioimageio.core.load_description_and_validate_format_only('affable-shark')
model

2024-05-09 16:58:45.301 | INFO     | bioimageio.spec._internal.io_utils:open_bioimageio_yaml:115 - 🦈 loading affable-shark 1 from https://uk1s3.embassy.ebi.ac.uk/public-datasets/bioimage.io/affable-shark/1/files/rdf.yaml


ValidationSummary(name='bioimageio validation', source_name='https://uk1s3.embassy.ebi.ac.uk/public-datasets/bioimage.io/affable-shark/1/files/rdf.yaml', status='passed', details=[ValidationDetail(name='initialized model 0.4.10', status='passed', errors=[], warnings=[]), ValidationDetail(name='bioimageio.spec format validation model 0.4.10', status='passed', errors=[], warnings=[WarningEntry(loc=('weights', 'pytorch_state_dict', 'dependencies'), msg='Custom dependencies (conda:environment.yaml) specified. Avoid this whenever possible to allow execution in a wider range of software environments.', type='warning', severity=30, severity_name='warning')])], env=[{'name': 'bioimageio.spec', 'version': '0.5.2post4'}])